# Spinful fermions

In [1]:
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch
from vmc_torch.experiment.tn_model import fTNModel
from vmc_torch.fermion_utils import fPEPS
import symmray as sr
import torch
import quimb.tensor as qtn


Lx = 2
Ly = 2
N_f = int(Lx*Ly)
n_fermions_per_spin = (Lx*Ly//2, Lx*Ly//2)
dtype = torch.float64

H = spinful_Fermi_Hubbard_square_lattice_torch(
    Lx, Ly, t=1.0, U=8.0, N_f=N_f, n_fermions_per_spin=None, no_u1_symmetry=True)
H_matrix = torch.tensor(H.to_dense(), dtype=dtype)

D = 4
flat = False
seed = 42
peps = sr.networks.PEPS_fermionic_rand(
    "Z2",
    Lx,
    Ly,
    D,
    phys_dim=[
        (0, 0),  # linear index 0 -> charge 0, offset 0
        (1, 1),  # linear index 1 -> charge 1, offset 1
        (1, 0),  # linear index 2 -> charge 1, offset 0
        (0, 1),  # linear index 3 -> charge 0, offset 1
    ],
    subsizes="equal",
    flat=flat,
    seed=seed,
)
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
peps = fPEPS(peps) # register as fPEPS object, which has get_amp function
fpeps_model = fTNModel(peps, max_bond=-1, dtype=dtype)
all_states = torch.tensor(H.hilbert.all_states())
psi_vec = fpeps_model(all_states)
E_vmc = torch.vdot(psi_vec, H_matrix @ psi_vec) / torch.vdot(psi_vec, psi_vec)
print("VMC energy:", E_vmc.item())



H_terms = sr.ham_fermi_hubbard_from_edges(
    symmetry='Z2',
    edges=tuple(peps.gen_bond_coos()),
    t=1.0,
    U=8.0,
    mu=0.0,
)
ham = qtn.LocalHam2D(Lx, Ly, H_terms)
E_double_layer = peps_np.compute_local_expectation_exact(ham.terms, normalized=True)
print("Double layer contraction energy:", E_double_layer)

/home/sijingdu/TNVMC/VMC_code/clean_symmray/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
F=2.16 C=3.35 S=4.00 P=6.32: 100%|██████████| 10/10 [00:00<00:00, 53.11it/s]


VMC energy: 9.850763601060661
Double layer contraction energy: 9.850763601060663


# Spinless fermions

In [2]:
from vmc_torch.hamiltonian_torch import spinless_Fermi_Hubbard_square_lattice_torch
from vmc_torch.experiment.tn_model import fTNModel
from vmc_torch.fermion_utils import fPEPS
import symmray as sr
import torch
import quimb.tensor as qtn


Lx = 2
Ly = 2
N_f = None
dtype = torch.float64

H = spinless_Fermi_Hubbard_square_lattice_torch(
    Lx, Ly, t=1.0, V=8.0, N_f=N_f, pbc=False)
H_matrix = torch.tensor(H.to_dense(), dtype=dtype)

D = 4
flat = False
seed = 42
peps = sr.networks.PEPS_fermionic_rand(
    "Z2",
    Lx,
    Ly,
    D,
    phys_dim=2,
    subsizes="equal",
    flat=flat,
    seed=seed,
)
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
peps = fPEPS(peps) # register as fPEPS object, which has get_amp function
fpeps_model = fTNModel(peps, max_bond=-1, dtype=dtype)
all_states = torch.tensor(H.hilbert.all_states())
psi_vec = fpeps_model(all_states)
E_vmc = torch.vdot(psi_vec, H_matrix @ psi_vec) / torch.vdot(psi_vec, psi_vec)
print("VMC energy:", E_vmc.item())


H_terms = sr.ham_fermi_hubbard_spinless_from_edges(
    symmetry='Z2',
    edges=tuple(peps.gen_bond_coos()),
    t=1.0,
    V=8.0,
)
ham = qtn.LocalHam2D(Lx, Ly, H_terms)
E_double_layer = peps_np.compute_local_expectation_exact(ham.terms, normalized=True)
print("Double layer contraction energy:", E_double_layer)

F=2.16 C=3.35 S=4.00 P=6.32: 100%|██████████| 10/10 [00:00<00:00, 814.08it/s]

VMC energy: 4.21656250122003
Double layer contraction energy: 4.2165625012200305


In [3]:
psi = peps_np.copy()
su = qtn.tensor_arbgeom_tebd.SimpleUpdateGen(
    psi, 
    ham,
    compute_energy_every=10,
    compute_energy_per_site=True,
)
for tau in [1.0, 0.3, 0.1, 0.01]:#, 0.1, 0.03, 0.01]:
    su.evolve(100, tau=tau)
psi_su = su.state
peps_su = su.get_state()
peps_su.equalize_norms_(value=1)
E_double_layer_su = peps_su.compute_local_expectation_exact(ham.terms, normalized=True)
print("Double layer contraction energy:", E_double_layer_su)

n=100, D=4, tau=1, max|dS|=0.171, energy≈0.27216: 100%|##########| 100/100 [00:01<00:00, 65.45it/s]  
n=200, D=4, tau=0.3, max|dS|=0.182, energy≈-0.213516: 100%|##########| 100/100 [00:01<00:00, 79.98it/s]  
n=300, D=4, tau=0.1, max|dS|=0.101, energy≈-0.280669: 100%|##########| 100/100 [00:01<00:00, 83.98it/s] 
n=400, D=4, tau=0.01, max|dS|=0.00913, energy≈-0.146575: 100%|##########| 100/100 [00:01<00:00, 84.86it/s]


Double layer contraction energy: -0.47169772597724874


In [4]:
peps_su.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
peps_su = fPEPS(peps_su) # register as fPEPS object, which has get_amp function
fpeps_model_su = fTNModel(peps_su, max_bond=-1, dtype=dtype)
all_states = torch.tensor(H.hilbert.all_states())
psi_vec_su = fpeps_model_su(all_states)
E_vmc_su = torch.vdot(psi_vec_su, H_matrix @ psi_vec_su) / torch.vdot(psi_vec_su, psi_vec_su)
print("VMC energy:", E_vmc_su.item())

F=2.16 C=3.35 S=4.00 P=6.32: 100%|██████████| 10/10 [00:00<00:00, 711.76it/s]

VMC energy: -0.4716977259772487
